In [16]:
!pip install pandas numpy matplotlib seaborn

In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

In [20]:
data = pd.read_excel('Coffee Shop Sales.xlsx')

In [23]:
# Renombramos las columnas de nuestra data.
bd = data.rename(columns={
    'transaction_id': 'id de transaccion',
    'transaction_date': 'fecha de transaccion',
    'transaction_time': 'hora de transaccion',
    'transaction_qty': 'cantidad vendida',
    'store_id': 'id tienda',
    'store_location': 'ubicacion tienda',
    'product_id': 'id producto',
    'unit_price': 'precio unitario',
    'product_category': 'categoria producto',
    'product_type': 'tipo de producto',
    'product_detail': 'detalles del producto'
})

# Creamos la nueva columna de ingreso.
bd['total de ingresos'] = bd['cantidad vendida'] * bd['precio unitario']

#Convertimos la 'fecha_de_transaccion' a un datetime para evitar errores más adelante.
bd['fecha de transaccion'] = pd.to_datetime(bd['fecha de transaccion'])

# Creamos un diccionario en español para los meses abreviados.
meses_español_abr = {
    1: 'ENE', 2: 'FEB', 3: 'MAR', 4: 'ABR', 5: 'MAY', 6: 'JUN',
    7: 'JUL', 8: 'AGO', 9: 'SEP', 10: 'OCT', 11: 'NOV', 12: 'DIC'
}

# Usamos el diccionario para modificar nuestra columna.
bd['mes abreviado'] = bd['fecha de transaccion'].dt.month.map(meses_español_abr).apply(lambda x: f'MES {x}')

bd['hora de transaccion'] = pd.to_datetime(bd['fecha de transaccion'].astype(str) + ' ' + bd['hora de transaccion'].astype(str))

#Creamos la columna hora y convertimos su formato.
bd['hora'] = bd['hora de transaccion'].dt.hour

bd['turno'] = pd.cut(bd['hora'],
                     bins=[0, 12, 24],  # Mañana: 0-12, Tarde/Noche: 12-24
                     labels=["HORARIO DE LA MAÑANA", "HORARIO DE LA TARDE/NOCHE"],
                     right=False,
                     ordered=False)

display(bd.head())

,id de transaccion,fecha de transaccion,hora de transaccion,cantidad vendida,id tienda,ubicacion tienda,id producto,precio unitario,categoria producto,tipo de producto,detalles del producto,total de ingresos,mes abreviado,hora,turno
0,1,2023-01-01,2023-01-01 07:06:11,2,5,Lower Manhattan,32,3.0,Coffee,Gourmet brewed coffee,Ethiopia Rg,6.0,MES ENE,7,HORARIO DE LA MAÑANA
1,2,2023-01-01,2023-01-01 07:08:56,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,6.2,MES ENE,7,HORARIO DE LA MAÑANA
2,3,2023-01-01,2023-01-01 07:14:04,2,5,Lower Manhattan,59,4.5,Drinking Chocolate,Hot chocolate,Dark chocolate Lg,9.0,MES ENE,7,HORARIO DE LA MAÑANA
3,4,2023-01-01,2023-01-01 07:20:24,1,5,Lower Manhattan,22,2.0,Coffee,Drip coffee,Our Old Time Diner Blend Sm,2.0,MES ENE,7,HORARIO DE LA MAÑANA
4,5,2023-01-01,2023-01-01 07:22:41,2,5,Lower Manhattan,57,3.1,Tea,Brewed Chai tea,Spicy Eye Opener Chai Lg,6.2,MES ENE,7,HORARIO DE LA MAÑANA


In [25]:
print("Valores únicos en la columna 'turno':")
print(bd['turno'].unique())
print("\nFrecuencia de cada turno:")
print(bd['turno'].value_counts())

Valores únicos en la columna 'turno':
['HORARIO DE LA MAÑANA', 'HORARIO DE LA TARDE/NOCHE']
Categories (2, object): ['HORARIO DE LA MAÑANA', 'HORARIO DE LA TARDE/NOCHE']

Frecuencia de cada turno:
turno
HORARIO DE LA MAÑANA         81751
HORARIO DE LA TARDE/NOCHE    67365
Name: count, dtype: int64


In [26]:
bd.info() #Evaluamos la nueva estructura de los datos de nuestro dataframe

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 149116 entries, 0 to 149115
Data columns (total 15 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id de transaccion      149116 non-null  int64         
 1   fecha de transaccion   149116 non-null  datetime64[ns]
 2   hora de transaccion    149116 non-null  datetime64[ns]
 3   cantidad vendida       149116 non-null  int64         
 4   id tienda              149116 non-null  int64         
 5   ubicacion tienda       149116 non-null  object        
 6   id producto            149116 non-null  int64         
 7   precio unitario        149116 non-null  float64       
 8   categoria producto     149116 non-null  object        
 9   tipo de producto       149116 non-null  object        
 10  detalles del producto  149116 non-null  object        
 11  total de ingresos      149116 non-null  float64       
 12  mes abreviado          149116 non-null  obje

In [33]:
# Desarrollamos la función para crear las estadísticas descriptivas condicionadas al ingreso.
def analizar_estadisticas(df, group_col):
    """
    Calcula estadísticas descriptivas para total_ingresos agrupados por una columna.

    Args:
        df: El DataFrame de entrada.
        group_col: La columna por la cual agrupar.

    Returns:
        Un DataFrame con estadísticas descriptivas.
    """
    if group_col not in df.columns:
        raise ValueError(f"Grouping column '{group_col}' not found in DataFrame.")

    # Calculamos las estadísticas descriptivas.
    resultado = df.groupby(group_col, observed=True).agg(
        transacciones=('id de transaccion', 'count'),
        ingreso_total=('total de ingresos', 'sum'),
        ingreso_minimo=('total de ingresos', 'min'),
        ingreso_maximo=('total de ingresos', 'max'),
        ingreso_promedio=('total de ingresos', 'mean'),
        ingreso_mediano=('total de ingresos', 'median'),
        desviacion_ingresos=('total de ingresos', 'std'),
        q1_ingresos=('total de ingresos', lambda x: x.quantile(0.25)),
        q3_ingresos=('total de ingresos', lambda x: x.quantile(0.75))
    )

    # Calculamos estadísticas adicionales.
    resultado['rango de ingresos'] = resultado['ingreso_maximo'] - resultado['ingreso_minimo']
    resultado['desviacion_mediana'] = df.groupby(group_col, observed=True)['total de ingresos'].apply(lambda x: np.sqrt(np.mean((x - np.median(x))**2)))

    # Coeficiente de variación
    resultado['CV % ingresos'] = (resultado['desviacion_ingresos'] / resultado['ingreso_promedio']) * 100

    # Redondeamos los valores a 2 decimales.
    resultado = resultado.round(2)

    return resultado

In [34]:
analisis_por_mes_abreviado = analizar_estadisticas(bd, 'mes abreviado')
meses_ordenados = ['MES ENE', 'MES FEB', 'MES MAR', 'MES ABR', 'MES MAY', 'MES JUN']
analisis_por_mes_abreviado = analisis_por_mes_abreviado.reindex(meses_ordenados)

display(analisis_por_mes_abreviado)

,transacciones,ingreso_total,ingreso_minimo,ingreso_maximo,ingreso_promedio,ingreso_mediano,desviacion_ingresos,q1_ingresos,q3_ingresos,rango de ingresos,desviacion_mediana,CV % ingresos
mes abreviado,,,,,,,,,,,,
MES ENE,17314,81677.74,0.8,360.0,4.72,3.75,4.95,3.0,6.0,359.2,5.05,104.99
MES FEB,16359,76145.19,0.8,45.0,4.65,3.75,3.02,3.0,6.0,44.2,3.15,64.90
MES MAR,21229,98834.68,0.8,72.0,4.66,3.75,3.08,3.0,6.0,71.2,3.21,66.14
MES ABR,25335,118941.08,0.8,360.0,4.69,3.75,4.39,3.0,6.0,359.2,4.49,93.41
MES MAY,33527,156727.76,0.8,360.0,4.67,3.75,4.08,3.0,6.0,359.2,4.18,87.26
MES JUN,35352,166485.88,0.8,360.0,4.71,3.75,4.89,3.0,6.0,359.2,4.98,103.75


In [35]:
analisis_por_ubicacion = analizar_estadisticas(bd, 'ubicacion tienda')
display(analisis_por_ubicacion)

,transacciones,ingreso_total,ingreso_minimo,ingreso_maximo,ingreso_promedio,ingreso_mediano,desviacion_ingresos,q1_ingresos,q3_ingresos,rango de ingresos,desviacion_mediana,CV % ingresos
ubicacion tienda,,,,,,,,,,,,
Astoria,50599,232243.91,0.8,45.0,4.59,3.75,2.88,3.0,6.0,44.2,3.00,62.69
Hell's Kitchen,50735,236511.17,0.8,360.0,4.66,3.75,5.84,3.0,6.0,359.2,5.91,125.26
Lower Manhattan,47782,230057.25,0.8,72.0,4.81,3.75,3.28,3.0,6.0,71.2,3.45,68.14


In [36]:
analisis_por_turno = analizar_estadisticas(bd, 'turno')
display(analisis_por_turno)

,transacciones,ingreso_total,ingreso_minimo,ingreso_maximo,ingreso_promedio,ingreso_mediano,desviacion_ingresos,q1_ingresos,q3_ingresos,rango de ingresos,desviacion_mediana,CV % ingresos
turno,,,,,,,,,,,,
HORARIO DE LA MAÑANA,81751,388288.67,0.8,360.0,4.75,3.75,5.13,3.0,6.0,359.2,5.22,107.96
HORARIO DE LA TARDE/NOCHE,67365,310523.66,0.8,45.0,4.61,3.75,2.76,3.0,6.0,44.2,2.89,59.94


In [37]:
analisis_por_categoria = analizar_estadisticas(bd, 'categoria producto')
display(analisis_por_categoria)

,transacciones,ingreso_total,ingreso_minimo,ingreso_maximo,ingreso_promedio,ingreso_mediano,desviacion_ingresos,q1_ingresos,q3_ingresos,rango de ingresos,desviacion_mediana,CV % ingresos
categoria producto,,,,,,,,,,,,
Bakery,22796,82315.64,2.65,9.00,3.61,3.50,0.58,3.25,3.75,6.35,0.59,16.10
Branded,747,13607.00,12.00,72.00,18.22,14.00,7.90,12.00,28.00,60.00,8.95,43.35
Coffee,58416,269952.45,2.00,17.00,4.62,4.20,2.00,3.00,6.00,15.00,2.05,43.36
Coffee beans,1753,40085.25,10.00,360.00,22.87,19.75,27.05,15.00,21.00,350.00,27.22,118.28
Drinking Chocolate,11468,72416.00,3.50,14.25,6.31,7.00,2.39,4.50,9.00,10.75,2.48,37.80
Flavours,6790,8408.80,0.80,3.20,1.24,1.60,0.44,0.80,1.60,2.40,0.57,35.13
Loose Tea,1210,11213.60,8.95,10.95,9.27,8.95,0.60,8.95,9.25,2.00,0.68,6.44
Packaged Chocolate,487,4407.64,6.40,13.33,9.05,7.60,2.87,7.60,13.33,6.93,3.21,31.72
Tea,45449,196405.95,2.50,12.00,4.32,5.00,1.68,2.55,6.00,9.50,1.81,38.90
